In [1]:
import sqlite3, datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import requests
import string
import time
import re
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials 

from bs4 import BeautifulSoup

pd.set_option('display.max_rows', 100000)
pd.set_option('display.max_columns', 100)

In [2]:
con = sqlite3.connect('database.sqlite')
scores = pd.read_sql('SELECT reviewid, artist, title, score, author FROM reviews', con)
genres = pd.read_sql('SELECT reviewid, genre FROM genres', con)
con.close()

# Preprocessing & Cleaning
### One Hot Encode Genres

In [3]:
genres['genre'] = genres['genre'].fillna('unknown')

In [4]:
genres['genre'].value_counts()

rock            9436
electronic      3874
unknown         2367
experimental    1815
rap             1559
pop/r&b         1432
metal            860
folk/country     685
jazz             435
global           217
Name: genre, dtype: int64

In [5]:
le = LabelEncoder()
genres['genre'] = le.fit_transform(genres['genre'])

In [6]:
genres_grouped = genres.groupby(['reviewid'])['genre'].apply(list).reset_index()
genres_grouped['genre'] = genres_grouped['genre'].apply(tuple)
genres_grouped['genre'] = [tuple(value) for value in genres_grouped['genre']]

In [7]:
mlb = MultiLabelBinarizer()
genres_ohe = mlb.fit_transform(genres_grouped['genre'].values.tolist())

In [8]:
genre_ohe_df = pd.DataFrame(genres_ohe, columns = le.classes_)
genre_ohe_df['reviewid'] = genres_grouped['reviewid'].values
genre_cols = ['reviewid', 'electronic', 'experimental', 'folk/country', 'global', 'jazz',
              'metal', 'pop/r&b', 'rap', 'rock', 'unknown']
genre_ohe_df = genre_ohe_df[genre_cols]

### Identify Top and Bottom 10% of Albums Based on Score

In [9]:
scores.shape

(18393, 5)

In [10]:
scores.head()

,reviewid,artist,title,score,author
0,22703,massive attack,mezzanine,9.3,nate patrin
1,22721,krallice,prelapsarian,7.9,zoe camp
2,22659,uranium club,all of them naturals,7.3,david glickman
3,22661,"kleenex, liliput",first songs,9.0,jenn pelly
4,22725,taso,new start,8.1,kevin lozano


In [11]:
# sort by ranking and extract the top and bottom 10% (1839) albums 
top_albums = scores.sort_values(by='score', ascending=False)[:1840].reset_index(drop=True)
bottom_albums = scores.sort_values(by='score', ascending=False)[-1839:].reset_index(drop=True)

### Clean Album and Artist Names

In [12]:
def clean_name(name):
    name = name.lower()
    name = re.sub('ö', 'o', name)
    name = re.sub(r'\([^)]*\)', '', name).strip()
    name = re.sub("[\(\[].*?[\)\]]", "", name).strip()
    name = re.sub(' & ', ' and ', name)
    name = re.sub('-', ' ', name)
    name = re.sub("[,.']", '', name)
    name = re.sub(' +', ' ', name)
    name = re.sub('"', '', name)
    name = re.sub('!', '', name)
    name = re.sub(':', '', name)
    name = re.sub('/', ' ', name)
    name = re.sub(r'[^0-9A-Za-z ]', '', name)
    name = name.strip()
    
    if name[-3:] == ' ep':
        name = name[:-3]
        
    if name[-4:] == ' ost':
        name = name[:-4]
        
    return name

In [13]:
top_albums['artist'] = top_albums['artist'].apply(clean_name)
top_albums['title'] = top_albums['title'].apply(clean_name)

bottom_albums['artist'] = bottom_albums['artist'].apply(clean_name)
bottom_albums['title'] = bottom_albums['title'].apply(clean_name)

In [14]:
# Clean up album names from review database

top_albums.loc[(top_albums['artist'] == 'the velvet underground') & 
               (top_albums['title'] == 'the velvet underground  45th anniversary super deluxe edition'), 
               'title'] = 'the velvet underground'

top_albums.loc[(top_albums['artist'] == 'roots manuva') &
               (top_albums['title'] == 'brand new secondhand'), 
               'title'] = 'brand new second hand'

top_albums.loc[(top_albums['artist'] == 'guided by voices') &
               (top_albums['title'] == 'human amusement at hourly rates: the best of guided by voices'), 
               'title'] = 'human amusements at hourly rates the best of guided by voices'

top_albums.loc[top_albums['title'] == 'tropiclia: a brazilian revolution in sound', 
               'title'] = 'tropicalia a brazilian revolution in sound'

top_albums.loc[top_albums['title'] == 'tigerbeat6 inc', 
               'title'] = 'tigerbeat inc disc 1'

top_albums.loc[(top_albums['artist'] == 'sufjan stevens') &
               (top_albums['title'] == 'greetings from michigan'), 
               'title'] = 'michigan'

top_albums.loc[(top_albums['artist'] == 'super furry animals') &
               (top_albums['title'] == 'love kraft'), 
               'title'] = 'lovekraft'

top_albums.loc[(top_albums['artist'] == 'david lang') &
               (top_albums['title'] == 'the little match girl passion'), 
               'title'] = 'lang the little match girl passion'

top_albums.loc[(top_albums['artist'] == 'ziq') &
               (top_albums['title'] == "tango n' vectif"), 
               'title'] = "tango n'vectif"

top_albums.loc[(top_albums['artist'] == 'william onyeabor') &
               (top_albums['title'] == 'who is william onyeabor'), 
               'title'] = 'world psychedelic classics 5 who is william onyeabor'

top_albums.loc[(top_albums['artist'] == 'justice') & 
               (top_albums['title'] == ''),
               'title'] = 'cross'

top_albums.loc[(top_albums['artist'] == 'love is all') & 
               (top_albums['title'] == 'nine times that same song'),
               'title'] = '9 times that same song'

top_albums.loc[top_albums['artist'] == 'prince the revolution',
               'artist'] = 'prince'

top_albums.loc[top_albums['title'] == 'dusk at cubist castle',
               'artist'] = 'the olivia tremor control'

top_albums.loc[top_albums['artist'] == 'sunn o',
               'artist'] = 'sunn 0)))'

top_albums.loc[top_albums['artist'] == 'bjrk',
               'artist'] = 'bjork'

top_albums.loc[top_albums['artist'] == 'sigur rs',
               'artist'] = 'sigur ros'

top_albums.loc[top_albums['artist'] == 'jhann jhannsson',
               'artist'] = 'jóhann jóhannsson'

top_albums.loc[(top_albums['artist'] == 'otis redding') &
               (top_albums['title'] == 'live at the whiskey a go go the complete recordings'),
               'title'] = 'live at the whisky a go go'

top_albums.loc[(top_albums['artist'] == 'half japanese') &
               (top_albums['title'] == 'gentlemen not beasts'),
               'title'] = 'half gentlemen not beasts'

top_albums.loc[(top_albums['artist'] == 'sigur ros') &
               (top_albums['title'] == 'agaetis byrjun'),
               'title'] = 'Ágaetis byrjun'

top_albums.loc[(top_albums['artist'] == 'various artists') &
               (top_albums['title'] == 'london is the place for me part four'),
               'title'] = 'london is the place for me 4'

top_albums.loc[(top_albums['artist'] == 'various artists') &
               (top_albums['title'] == 'black foliage animation music vol 1'),
               'artist'] = 'the olivia tremor control'

top_albums.loc[(top_albums['artist'] == 'various artists') &
               (top_albums['title'] == 'wattstax'),
               'title'] = 'wattstax: the living word'

top_albums.loc[(top_albums['artist'] == 'various artists') &
               (top_albums['title'] == 'music from the unrealized film script dusk at cubist castle'),
               'title'] = 'dusk at cubist castle'

top_albums.loc[(top_albums['artist'] == 'various artists') &
               (top_albums['title'] == 'dusk at cubist castle'),
               'artist'] = 'olivia tremor control'

top_albums.loc[(top_albums['artist'] == 'various artists') &
               (top_albums['title'] == 'de stijl'),
               'artist'] = 'the white stripes'

top_albums.loc[(top_albums['artist'] == 'various artists') &
               (top_albums['title'] == 'nigeria 70 the definitive story of 1970s funky lagos'),
               'title'] = 'nigeria 70 funky lagos'

top_albums.loc[(top_albums['artist'] == 'various artists') &
               (top_albums['title'] == 'methodology 74 78 attic tapes'),
               'artist'] = 'cabaret voltaire'

top_albums.loc[(top_albums['artist'] == 'various artists') &
               (top_albums['title'] == 'shangaan electro new wave dance music from south africa'),
               'title'] = 'new wave dance music from south africa'

top_albums.loc[(top_albums['artist'] == 'various artists') &
               (top_albums['title'] == 'new wave dance music from south africa'),
               'artist'] = 'shangaan electro'

top_albums.loc[(top_albums['artist'] == 'various artists') &
               (top_albums['title'] == 'brazil 70 after tropiclia new directions in brazilian music in the 1970s'),
               'title'] = 'brazil 70 after tropicalia'

top_albums.loc[(top_albums['artist'] == 'morton feldman ives ensemble') &
               (top_albums['title'] == 'string quartet ii'),
               'title'] = 'morton feldman string quartet (ii)'

top_albums.loc[top_albums['artist'] == 'orchestre poly rythmo de cotonou',
               'artist'] = 'tp orchestre poly rythmo'

top_albums.loc[(top_albums['artist'] == 'tp orchestre poly rythmo') &
               (top_albums['title'] == 'volume one the vodoun effect'),
               'title'] = 'the vodoun effect'

top_albums.loc[top_albums['artist'] == 'gyrgy ligeti',
               'artist'] = 'gyorgy ligeti'

top_albums.loc[(top_albums['artist'] == 'gyorgy ligeti') &
               (top_albums['title'] == 'the ligeti project ii'),
               'title'] = 'Ligeti Project Vol.2 - Lontano, Atmosphères, Apparitions, San Francisco Polyphony & Concert Românesc'

top_albums.loc[top_albums['artist'] == 'caf tacuba',
               'artist'] = 'café tacvba'

top_albums.loc[top_albums['artist'] == 'brian eno karl hyde',
               'artist'] = 'eno • hyde'

top_albums.loc[(top_albums['artist'] == 'peter brtzmann william parker hamid drake') &
               (top_albums['title'] == 'never too late but always too early'),
               'title'] = 'peter brotzmann'

top_albums.loc[top_albums['artist'] == 'fantmas',
               'artist'] = 'fantomas'

top_albums.loc[top_albums['artist'] == 'world standard wechsel garland',
               'artist'] = 'world standard'

top_albums.loc[(top_albums['artist'] == 'various artists') &
               (top_albums['title'] == 'young money rise of an empire'),
               'artist'] = 'young money'

top_albums.loc[(top_albums['artist'] == 'young money') &
               (top_albums['title'] == 'young money rise of an empire'),
               'title'] = 'rise of an empire'

top_albums.loc[top_albums['artist'] == 'koenji hyakkei',
               'artist'] = 'koenjihyakkei'

top_albums.loc[top_albums['artist'] == 'the peter brtzmann sextet   quartet',
               'artist'] = 'the peter brotzmann sextet'


In [15]:
top_albums_remove = [
    'matador at 21',
    'run the road',
    'the art of field recording vol 1',
    'rwd magazine mixtape vol 1',
    'now thing',
    'yellow pills',
    'the rise and fall of paramount records volume one',
    'the rise and fall of paramount records volume two',
    'steam kodok 26 a go go ultrarities from the sixties singapore and south east asia underground',
    'the art of field recording vol 2',
    'stones throw 101',
    'give me love songs of the brokenhearted baghdad 1925 1929'
    'stax 50th anniversary',
    'the third unheard connecticut hip hop 1979 1983',
    'japanese independent music',
    'the upsetter selection a lee perry jukebox',
    'this may be my last time singing',
    'john peel a tribute',
    'parallelogram' ,
]

In [16]:
for album in top_albums_remove:
    top_albums = top_albums.loc[top_albums.title != album]
    
top_albums = top_albums.reset_index()

In [17]:
bottom_albums.loc[(bottom_albums['artist'] == 'triangle'), 
                  'title'] = 'star'

bottom_albums.loc[(bottom_albums['artist'] == 'does it offend you, yeah?') &
                  (bottom_albums['title'] == 'you have no idea what you are getting yourself into'), 
                  'title'] = 'you have no idea what youre getting yourself into'

bottom_albums.loc[(bottom_albums['artist'] == 'the moldy peaches') &
                  (bottom_albums['title'] == 'moldy peaches 2000: unreleased cutz and live jamz'), 
                  'title'] = 'unreleased cutz and live jamz'

bottom_albums.loc[(bottom_albums['artist'] == 'ghostland observatory') &
                  (bottom_albums['title'] == 'robotique majestic'), 
                  'title'] = 'robotique majestique'

bottom_albums.loc[(bottom_albums['artist'] == 'mia') &
                  (bottom_albums['title'] == 'y'), 
                  'title'] = '/ \\ / \\ / \\ y / \\'

bottom_albums = bottom_albums.drop(402, axis=0).reset_index(drop=True)

bottom_albums.loc[bottom_albums['artist'] == 'bjrk',
                   'artist'] = 'bjork'

bottom_albums.loc[(bottom_albums['artist'] == 'various artists') &
                  (bottom_albums['title'] == 'thumbsucker'), 
                  'artist'] = 'tim delaughter'

bottom_albums.loc[(bottom_albums['artist'] == 'various artists') &
                  (bottom_albums['title'] == 'guilt by association'), 
                  'title'] = 'guilt by association digital only bounus version'

bottom_albums.loc[(bottom_albums['artist'] == 'various artists') &
                  (bottom_albums['title'] == 'tron legacy reconfigured'), 
                  'artist'] = 'daft punk'

bottom_albums.loc[(bottom_albums['artist'] == 'various artists') &
                  (bottom_albums['title'] == 'songs from wonderland'), 
                  'artist'] = 'original cast of wonderland'

bottom_albums.loc[(bottom_albums['artist'] == 'various artists') &
                  (bottom_albums['title'] == 'discovered a collection of daft funk samples'), 
                  'title'] = 'daft punk discovered a collection of daft funk samples'

bottom_albums.loc[(bottom_albums['artist'] == 'various artists') &
                  (bottom_albums['title'] == 'daft punk discovered a collection of daft funk samples'), 
                  'artist'] = 'daft punk'

bottom_albums.loc[(bottom_albums['artist'] == 'various artists') &
                  (bottom_albums['title'] == 'kitsun maison 9'), 
                  'title'] = 'kitsun maison compilation 9 petit bateau edition'

bottom_albums.loc[(bottom_albums['artist'] == 'various artists') &
                  (bottom_albums['title'] == 'celebrating the music of inside llewyn davis'), 
                  'title'] = 'inside llewyn davis'

bottom_albums.loc[(bottom_albums['artist'] == 'various artists') &
                  (bottom_albums['title'] == 'army of me remixes and covers'), 
                  'artist'] = 'bjork'

bottom_albums.loc[bottom_albums['artist'] == 'charlatans uk',
                   'artist'] = 'the charlatans'

bottom_albums.loc[(bottom_albums['artist'] == 'dave grohl') &
                  (bottom_albums['title'] == 'sound city real to reel'), 
                  'artist'] = 'sound city real to reel'

bottom_albums.loc[(bottom_albums['artist'] == 'the visible men') &
                  (bottom_albums['title'] == 'love 30'), 
                  'title'] = 'Love:30'

bottom_albums.loc[(bottom_albums['artist'] == 'dabrye') &
                  (bottom_albums['title'] == 'two three'), 
                  'title'] = 'Two / Three'

bottom_albums.loc[(bottom_albums['artist'] == 'the hidden cameras') &
                  (bottom_albums['title'] == 'origin orphan'), 
                  'title'] = 'Origin:Orphan'

bottom_albums.loc[(bottom_albums['artist'] == 'the flaming lips stardeath and white dwarfs') &
                  (bottom_albums['title'] == 'the dark side of the moon'), 
                  'title'] = 'The Flaming Lips and Stardeath and White Dwarfs with Henry Rollins and Peaches Doing The Dark Side of the Moon'

bottom_albums.loc[bottom_albums['artist'] == 'beholdthe arctopus',
                   'artist'] = 'behold... the arctopus'

bottom_albums.loc[(bottom_albums['artist'] == 'auteurs') &
                  (bottom_albums['title'] == 'das capital the songwriting genius of luke haines and the auteurs'), 
                  'artist'] = 'luke haines the auteurs'

bottom_albums.loc[(bottom_albums['artist'] == 'luke haines the auteurs') &
                  (bottom_albums['title'] == 'das capital the songwriting genius of luke haines and the auteurs'), 
                  'title'] = 'das capital - the songwriting genius of luke haines and the auteurs'

bottom_albums.loc[(bottom_albums['artist'] == 'cibelle') &
                  (bottom_albums['title'] == 'las vnus resort palace hotel'), 
                  'title'] = 'las venus resort palace hotel'

bottom_albums.loc[(bottom_albums['artist'] == 'british sea power') &
                  (bottom_albums['title'] == 'zeus'), 
                  'title'] = 'zeus ep'

bottom_albums.loc[(bottom_albums['artist'] == 'various artists') &
                   (bottom_albums['title'] == 'young money rise of an empire'),
                   'artist'] = 'young money'

bottom_albums.loc[(bottom_albums['artist'] == 'young money') &
                   (bottom_albums['title'] == 'young money rise of an empire'),
                   'title'] = 'rise of an empire'

bottom_albums.loc[(bottom_albums['artist'] == 'the mars volta') &
                   (bottom_albums['title'] == 'de loused in the comatorium'),
                   'title'] = 'deloused in the comatorium'

bottom_albums.loc[(bottom_albums['artist'] == 'ilya monosov') &
                   (bottom_albums['title'] == 'seven lucky plays or how to fix songs for a broken heart'),
                   'artist'] = 'ilya e. monosov'

bottom_albums.loc[bottom_albums['artist'] == 'takagi masakatsu',
                   'artist'] = 'masakatsu takagi'

bottom_albums.loc[bottom_albums['artist'] == 'steve buscemi elliott sharp',
                   'artist'] = 'steve buscemi & elliott sharp'

bottom_albums.loc[bottom_albums['artist'] == 'sarah lee guthrie johnny irion',
                   'artist'] = 'sarah lee guthrie and johnny irion'

bottom_albums.loc[(bottom_albums['artist'] == 'sarah lee guthrie and johnny irion') &
                   (bottom_albums['title'] == 'wassiac way'),
                   'title'] = 'wassaic way'

bottom_albums.loc[(bottom_albums['artist'] == 'exhaust') &
                   (bottom_albums['title'] == 'enregistraur'),
                   'title'] = 'enregistreur'

bottom_albums.loc[bottom_albums['artist'] == 'mats gustafsson colin stetson',
                   'artist'] = 'colin stetson mats gustafsson'

bottom_albums.loc[bottom_albums['artist'] == 'miss kittin and the hacker',
                   'artist'] = 'miss kittin the hacker'

bottom_albums.loc[(bottom_albums['artist'] == 'electric six') &
                   (bottom_albums['title'] == 'seor smoke'),
                   'title'] = 'senor smoke'

bottom_albums.loc[(bottom_albums['artist'] == 'unkle') &
                   (bottom_albums['title'] == 'end titlesstories for film'),
                   'title'] = 'end titles... stories for film'

bottom_albums.loc[bottom_albums['artist'] == 'jackson and his computerband',
                   'artist'] = 'jackson and his computer band'

bottom_albums.loc[(bottom_albums['artist'] == 'lemon jelly') &
                   (bottom_albums['title'] == 'lemonjellyky'),
                   'title'] = 'lemon jelly.ky'

bottom_albums.loc[(bottom_albums['artist'] == 'earl zinger') &
                   (bottom_albums['title'] == 'put your phazers on stun throw your health food skyward'),
                   'title'] = 'put your phasers on ...'

bottom_albums.loc[(bottom_albums['artist'] == 'stereo image') &
                   (bottom_albums['title'] == 'stereoimage'),
                   'title'] = 's/t'

bottom_albums.loc[(bottom_albums['artist'] == 'adrian orange and her band') &
                   (bottom_albums['title'] == 'adrian orange and her band'),
                   'artist'] = 'adrian orange, her band'

bottom_albums.loc[(bottom_albums['artist'] == 'suntanama') &
                   (bottom_albums['title'] == 'another'),
                   'artist'] = 'the suntanama'

bottom_albums.loc[(bottom_albums['artist'] == 'josh rouse') &
                   (bottom_albums['title'] == 'subttulo'),
                   'title'] = 'subtitulo'

bottom_albums.loc[(bottom_albums['artist'] == 'the peppermints') &
                   (bottom_albums['title'] == 'jess chryst'),
                   'title'] = 'jesus chryst'

bottom_albums.loc[(bottom_albums['artist'] == 'they shoot horses dont they') &
                   (bottom_albums['title'] == 'pick up sticks'),
                   'title'] = 'pickup sticks'

bottom_albums.loc[(bottom_albums['artist'] == 'marky ramone the speedkings') &
                   (bottom_albums['title'] == 'legends bleed'),
                   'artist'] = 'marky ramone and the speedkings'

bottom_albums.loc[(bottom_albums['artist'] == 'youngblood brass band') &
                   (bottom_albums['title'] == 'centerlevelroar'),
                   'artist'] = 'center:level:roar'

bottom_albums.loc[(bottom_albums['artist'] == 'boris ian astbury') &
                   (bottom_albums['title'] == 'bxi'),
                   'artist'] = 'boris & ian astbury'

bottom_albums.loc[(bottom_albums['artist'] == 'liliput kleenex') &
                   (bottom_albums['title'] == 'live recordings tv clips and roadmovie'),
                   'artist'] = 'kleenex/liliput'

bottom_albums.loc[(bottom_albums['artist'] == 'kleenex/liliput') &
                   (bottom_albums['title'] == 'live recordings tv clips and roadmovie'),
                   'title'] = 'live recordings, tv-clips & roadmovie'

bottom_albums.loc[bottom_albums['artist'] == 'kleenexgirlwonder',
                   'artist'] = 'kleenex girl wonder'

bottom_albums.loc[(bottom_albums['artist'] == 'vert') &
                   (bottom_albums['title'] == 'some beans and an octopus'),
                   'artist'] = 'some beans and an octopus, vert'

bottom_albums.loc[bottom_albums['artist'] == 'murder city devils',
                   'artist'] = 'the murder city devils'

bottom_albums.loc[(bottom_albums['artist'] == 'the murder city devils') &
                   (bottom_albums['title'] == 'rip'),
                   'artist'] = 'r.i.p.'

bottom_albums.loc[(bottom_albums['artist'] == 'snoop dogg') &
                   (bottom_albums['title'] == 'rg  the masterpiece'),
                   'title'] = 'r&g (rhythm & gangsta): the masterpiece'

bottom_albums.loc[(bottom_albums['artist'] == 'pixies') &
                   (bottom_albums['title'] == 'ep 2'),
                   'title'] = 'ep2'

bottom_albums.loc[(bottom_albums['artist'] == 'thurston moore') &
                   (bottom_albums['title'] == 'sensitive lethal'),
                   'title'] = 'sensitive/lethal'

bottom_albums.loc[bottom_albums['artist'] == 'muggs',
                   'artist'] = 'dj muggs'

bottom_albums.loc[bottom_albums['artist'] == 's   s   s',
                   'artist'] = 's/s/s'

bottom_albums.loc[(bottom_albums['artist'] == 'architecture in helsinki') &
                   (bottom_albums['title'] == 'now  4eva'),
                   'title'] = 'now + 4eva'

In [18]:
bottom_albums_remove = [
    'woodstock 40 years on back to yasgurs farm',
    'zang tumb tuum the ztt box set',
    'listen to what the man said popular artists pay tribute to paul mccartney',
    'clicks and cuts',
    'to spirit back the mews',
    'definitive jux presents iii',
    'catch the throne the mixtape',
    'rave on a tribute to buddy holly',
    'johnny cash remixed',
    'bbc radio 1 established 1967'
]

In [19]:
for album in bottom_albums_remove:
    bottom_albums = bottom_albums.loc[bottom_albums.title != album]
    
bottom_albums = bottom_albums.reset_index()

In [20]:
playlist_search_albums = [
    'no thanks the 70s punk rebellion',
    'in the beginning there was rhythm',
    'ultimate breaks and beats the complete collection',
    'the very best of ethiopiques hypnotic grooves from the legendary series',
    'the big stiff box set',
    'miami sound rare funk and soul from miami florida 1967 1974',
    'journey into paradise the larry levan story',
    'can you dig it the music and politics of black action films 1968-75',
    'children of nuggets original artyfacts from the second psychedelic era 1976-1996',
    'music from the oc mix 5',
    'american hardcore the history of american punk rock 1980 1986'
]

# Scrape Album Song Data From Spotify
### Import Libraries and Enter Credentials
Note:  Delete Spotify Client ID and Client Secret Before Committing

In [21]:
client_id = '122bc2fea97c4c96a4cfeacea74a9883'
client_secret = 'c1dc56a9f8374ad7a222e3d958ea4cce'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

### Define Helper Functions and Prepare Dataframe for Scraped Data

In [22]:
# Define columns for scraped data

df_columns = ['reviewid', 'artist_name', 'album_title', 'score', 'song_name', 'song_id', 'song_track_number', 
              'song_duration', 'song_explicit', 'song_uri', 'acousticness', 'danceability', 'energy', 'key', 
              'mode', 'time_signature', 'instrumentalness', 'liveness', 'speechiness', 'tempo', 'valence', 
              'popularity', 'release_date']

In [23]:
def check_search_counter(search_counter):
    if search_counter == 4:
        time.sleep(np.random.uniform(2,4))
        return 0
    else:
        return search_counter + 1

In [24]:
def identify_album_matches(album_search_items, album_title, artist_name, search_index, album_list, uri_list):
    
    album_search_result = clean_name(album_search_items[search_index]['name'])
    artist_search_result = clean_name(album_search_items[search_index]['artists'][0]['name'])

    if album_search_result in clean_name(album_title) or clean_name(album_title) in album_search_result:
        if clean_name(artist_name) in artist_search_result or artist_search_result in clean_name(artist_name):
            uri_list.append(album_search_items[search_index]['uri'])
            album_list.append(artist_search_result)
    
    return uri_list, album_list

In [25]:
def identify_playlist_matches(playlist_search_items, playlist_title, artist_name, search_index, album_list, uri_list):
    
    playlist_search_result = clean_name(playlist_search_items[search_index]['name'])

    if playlist_search_result in playlist_title or playlist_title in playlist_search_result:
        uri_list.append(playlist_search_items[search_index]['uri'])
        album_list.append(playlist_search_result)
    
    return uri_list, album_list

In [26]:
def scrape_album_songs(df):
    compiled_df = pd.DataFrame(columns=df_columns)
    missing_album_indices = []
    start_time = time.time()
    search_counter = 0

    for album_index in range(len(df)):
        
        if album_index % 5 == 0:
            print('')
            print('Albums Completed: ', album_index)
            print('Elapsed Time: ', time.time() - start_time)
            print('')

        
        sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=60)
                
        reviewid = df.iloc[album_index]['reviewid']
        artist_name = df.iloc[album_index]['artist']
        album_title = df.iloc[album_index]['title']
        score = df.iloc[album_index]['score']
        
        # search for playlists
        if album_title in playlist_search_albums:
            
            search_results = sp.search(album_title, limit='50', type='playlist')
            playlist_items = search_results['playlists']['items']

            playlist_uris = []
            playlist_names = []

            for playlist_search_index in range(len(playlist_items)):
                playlist_uris, playlist_names = identify_playlist_matches(playlist_items, album_title, artist_name, 
                                                                 playlist_search_index, playlist_names, playlist_uris)

            if len(playlist_uris) == 0:
                print('Album and Playlist Not Found: ', artist_name, ' - ', album_title)
                missing_album_indices.append(album_index)
            else:
                playlist_data = sp.playlist_tracks(playlist_uris[0])

                for song_index in range(len(playlist_data['items'])):
                    song_name = playlist_data['items'][song_index]['track']['name']
                    song_id = playlist_data['items'][song_index]['track']['id']
                    song_track_number = playlist_data['items'][song_index]['track']['track_number']
                    song_duration = playlist_data['items'][song_index]['track']['duration_ms']
                    song_explicit = playlist_data['items'][song_index]['track']['explicit']
                    song_uri = playlist_data['items'][song_index]['track']['uri']

                    track_features = sp.audio_features(song_uri)

                    if track_features != [None]:
                        acousticness = track_features[0]['acousticness']
                        danceability = track_features[0]['danceability']
                        energy = track_features[0]['energy']
                        key = track_features[0]['key']
                        mode = track_features[0]['mode']
                        time_signature = track_features[0]['time_signature']
                        instrumentalness = track_features[0]['instrumentalness']
                        liveness = track_features[0]['liveness']
                        loudness = track_features[0]['loudness']
                        speechiness = track_features[0]['speechiness']
                        tempo = track_features[0]['tempo']
                        valence = track_features[0]['valence']
                        popularity = sp.track(song_uri)['popularity']
                        release_date = sp.track(song_uri)['album']['release_date']


                    new_row = [reviewid, artist_name, album_title, score, song_name, song_id, song_track_number,
                               song_duration, song_explicit, song_uri, acousticness, danceability, energy,
                               key, mode, time_signature, instrumentalness, liveness, speechiness, tempo, 
                               valence, popularity, release_date]

                    compiled_df.loc[len(compiled_df)] = new_row

        
        # search for albums
        else:
            search_results = sp.search(album_title, limit='50', type='album')
            search_counter = check_search_counter(search_counter)

            # iterate through album search results and extract album uris that match artist name

            album_uris = []
            album_names = []
            album_items = search_results['albums']['items']

            for album_search_index in range(len(album_items)):

                album_uris, album_names = identify_album_matches(album_items, album_title, artist_name, 
                                                                 album_search_index, album_names, album_uris)

            # if no albums are found, begin searching truncated album names            

            if len(album_uris) == 0 and len(album_title.split(' ')) > 1:
                truncated_album_title = album_title

                while len(truncated_album_title.split(' ')) > 1:
                    truncated_album_title = ' '.join(truncated_album_title.split(' ')[:-1])
                    search_results = sp.search(truncated_album_title, limit='50', type='album')
                    search_counter = check_search_counter(search_counter)
                    album_items = search_results['albums']['items']

                    for album_search_index in range(len(album_items)):
                        album_uris, album_names = identify_album_matches(album_items, truncated_album_title, 
                                                                         artist_name, album_search_index, 
                                                                         album_names, album_uris)

            # if no albums are found after truncated search, attempt to search by artist name

            if len(album_uris) == 0:
                search_results = sp.search(artist_name, limit='50', type='artist')
                search_counter = check_search_counter(search_counter)
                artist_items = search_results['artists']['items']
                artist_uri = None

                for artist_index in range(len(artist_items)):
                    artist_search_result = clean_name(artist_items[artist_index]['name'])
                    if artist_search_result == artist_name:
                        artist_uri = artist_items[artist_index]['uri']
                        break

                if artist_uri != None: 
                    artist_albums = sp.artist_albums(artist_uri, album_type='album')
                    arist_album_items = artist_albums['items']

                    for album_search_index in range(len(arist_album_items)):
                        album_uris, album_names = identify_album_matches(arist_album_items, album_title, 
                                                                         artist_name, album_search_index, 
                                                                         album_names, album_uris)
                        
            # if no albums are found after artist name search, attempt to search by artist name + album name

            if len(album_uris) == 0:
                search_results = sp.search(artist_name + ' ' + album_title, limit='50', type='album')
                search_counter = check_search_counter(search_counter)
                
                album_uris = []
                album_names = []
                album_items = search_results['albums']['items']

                for album_search_index in range(len(album_items)):

                    album_uris, album_names = identify_album_matches(album_items, album_title, artist_name, 
                                                                     album_search_index, album_names, album_uris)

            # if no albums are found after truncated search, flag album as not identified and record index

            if len(album_uris) == 0:
                print('Album Not Found: ', artist_name, ' - ', album_title)
                missing_album_indices.append(album_index)
            else:
                album_data = sp.album_tracks(album_uris[0])

                for song_index in range(len(album_data['items'])):
                    song_name = album_data['items'][song_index]['name']
                    song_id = album_data['items'][song_index]['id']
                    song_track_number = album_data['items'][song_index]['track_number']
                    song_duration = album_data['items'][song_index]['duration_ms']
                    song_explicit = album_data['items'][song_index]['explicit']
                    song_uri = album_data['items'][song_index]['uri']

                    track_features = sp.audio_features(song_uri)

                    if track_features != [None]:
                        acousticness = track_features[0]['acousticness']
                        danceability = track_features[0]['danceability']
                        energy = track_features[0]['energy']
                        key = track_features[0]['key']
                        mode = track_features[0]['mode']
                        time_signature = track_features[0]['time_signature']
                        instrumentalness = track_features[0]['instrumentalness']
                        liveness = track_features[0]['liveness']
                        loudness = track_features[0]['loudness']
                        speechiness = track_features[0]['speechiness']
                        tempo = track_features[0]['tempo']
                        valence = track_features[0]['valence']
                        popularity = sp.track(song_uri)['popularity']
                        release_date = sp.track(song_uri)['album']['release_date']


                    new_row = [reviewid, artist_name, album_title, score, song_name, song_id, song_track_number,
                               song_duration, song_explicit, song_uri, acousticness, danceability, energy,
                               key, mode, time_signature, instrumentalness, liveness, speechiness, tempo, 
                               valence, popularity, release_date]

                    compiled_df.loc[len(compiled_df)] = new_row  
    
    return compiled_df, missing_album_indices

In [27]:
top_album_songs, missing_top_album_indices = scrape_album_songs(top_albums)


Albums Completed:  0
Elapsed Time:  0.0002522468566894531


Albums Completed:  5
Elapsed Time:  18.175711154937744


Albums Completed:  10
Elapsed Time:  43.83176517486572


Albums Completed:  15
Elapsed Time:  71.35407209396362

Album Not Found:  the beatles  -  stereo box

Albums Completed:  20
Elapsed Time:  85.77578711509705


Albums Completed:  25
Elapsed Time:  140.48061108589172


Albums Completed:  30
Elapsed Time:  199.83423614501953


Albums Completed:  35
Elapsed Time:  226.133394241333


Albums Completed:  40
Elapsed Time:  247.88977313041687

Album Not Found:  public image ltd  -  metal box

Albums Completed:  45
Elapsed Time:  272.1628441810608


Albums Completed:  50
Elapsed Time:  305.2127161026001


Albums Completed:  55
Elapsed Time:  327.32496428489685


Albums Completed:  60
Elapsed Time:  346.34081411361694


Albums Completed:  65
Elapsed Time:  384.7443652153015


Albums Completed:  70
Elapsed Time:  404.3264961242676


Albums Completed:  75
Elapsed Time:  421.56


Albums Completed:  485
Elapsed Time:  2817.032560110092

Album Not Found:  jim orourke  -  im happy and im singing and a 1 2 3 4

Albums Completed:  490
Elapsed Time:  2844.1479432582855


Albums Completed:  495
Elapsed Time:  2865.311786174774

Album Not Found:  liliput  -  complete recordings

Albums Completed:  500
Elapsed Time:  2901.230882167816


Albums Completed:  505
Elapsed Time:  2921.4980380535126

Album Not Found:  the breeders  -  lsxx
Album Not Found:  this mortal coil  -  hdcd box set

Albums Completed:  510
Elapsed Time:  2946.4662840366364

Album Not Found:  matthew herbert  -  secondhand sounds

Albums Completed:  515
Elapsed Time:  2959.422662258148

Album Not Found:  enduser  -  15 tracks
Album and Playlist Not Found:  various artists  -  miami sound rare funk and soul from miami florida 1967 1974

Albums Completed:  520
Elapsed Time:  2975.48659825325

Album Not Found:  art bears  -  art box
Album Not Found:  area  -  revolution

Albums Completed:  525
Elapsed Tim

Album Not Found:  mike cooper  -  trout steel
Album Not Found:  kid creole  -  going places the august darnell years 1976 1983

Albums Completed:  920
Elapsed Time:  4870.108125209808

Album Not Found:  teenage fanclub  -  four thousand seven hundred and sixty six seconds a shortcut to teenage fanclub

Albums Completed:  925
Elapsed Time:  4900.883288145065


Albums Completed:  930
Elapsed Time:  4935.968740224838


Albums Completed:  935
Elapsed Time:  4957.4960832595825

Album Not Found:  melchior productions  -  no disco future

Albums Completed:  940
Elapsed Time:  4976.190840244293

Album Not Found:  various artists  -  black mirror reflections in global music

Albums Completed:  945
Elapsed Time:  5002.567109107971


Albums Completed:  950
Elapsed Time:  5023.48991727829

Album Not Found:  various artists  -  honest strings a tribute to the life and work of jack rose
Album Not Found:  fuck buttons  -  street horrrsing
Album Not Found:  john zorn  -  songs from the hermetic theatr


Albums Completed:  1310
Elapsed Time:  6709.909705400467


Albums Completed:  1315
Elapsed Time:  6728.430806159973

Album Not Found:  viet cong  -  viet cong

Albums Completed:  1320
Elapsed Time:  6745.983223199844

Album Not Found:  fennesz  -  live in japan
Album Not Found:  thalia zedek  -  been here and gone

Albums Completed:  1325
Elapsed Time:  6764.019954204559


Albums Completed:  1330
Elapsed Time:  6786.792686223984


Albums Completed:  1335
Elapsed Time:  6812.885332107544


Albums Completed:  1340
Elapsed Time:  6835.62944316864


Albums Completed:  1345
Elapsed Time:  6849.833096265793

Album Not Found:  gloss  -  trans day of revenge

Albums Completed:  1350
Elapsed Time:  6877.035192251205

Album Not Found:  diplo mia  -  piracy funds terrorism vol 1

Albums Completed:  1355
Elapsed Time:  6916.37283205986

Album Not Found:  john cale  -  sun blindness music

Albums Completed:  1360
Elapsed Time:  6941.164806127548


Albums Completed:  1365
Elapsed Time:  6959.130791


Albums Completed:  1740
Elapsed Time:  8760.316863059998


Albums Completed:  1745
Elapsed Time:  8788.83197426796

Album Not Found:  resplendent  -  wings beneath the sills casket city wolves
Album Not Found:  microwaves  -  system 2

Albums Completed:  1750
Elapsed Time:  8809.411799192429

Album Not Found:  various artists  -  baby how can it be songs of love lust and contempt from the 1920s and 1930s

Albums Completed:  1755
Elapsed Time:  8837.353934049606

Album Not Found:  the books  -  a dot in time

Albums Completed:  1760
Elapsed Time:  8855.663977146149


Albums Completed:  1765
Elapsed Time:  8889.904689311981


Albums Completed:  1770
Elapsed Time:  8905.951830148697


Albums Completed:  1775
Elapsed Time:  8935.442682266235


Albums Completed:  1780
Elapsed Time:  8952.745224237442


Albums Completed:  1785
Elapsed Time:  8986.373197078705

Album Not Found:  low  -  one more reason to forget
Album Not Found:  jean grae  -  jeanius

Albums Completed:  1790
Elapsed Time:  

In [190]:
print('Found {}% of Top 10% Pitchfork Albums on Spotify'.format(round(100-len(missing_top_album_indices)/1839*100, 2)))

Found 87.98% of Top 10% Pitchfork Albums on Spotify


In [29]:
len(top_albums) - len(missing_top_album_indices)

1602

### Scrape Song Info For Bottom 10% of Albums

In [30]:
bottom_album_songs, missing_bottom_album_indices = scrape_album_songs(bottom_albums)


Albums Completed:  0
Elapsed Time:  0.0007050037384033203


Albums Completed:  5
Elapsed Time:  19.451534032821655

Album Not Found:  spaceghostpurrp  -  bmw
Album Not Found:  metal urbain  -  anarchy in paris

Albums Completed:  10
Elapsed Time:  33.60040211677551

Album Not Found:  tortoise bonnie prince billy  -  the brave and the bold

Albums Completed:  15
Elapsed Time:  54.824968099594116

Album Not Found:  the red crayola  -  soldier talk

Albums Completed:  20
Elapsed Time:  76.04783201217651


Albums Completed:  25
Elapsed Time:  100.95314908027649


Albums Completed:  30
Elapsed Time:  121.53869295120239

Album Not Found:  the libertines  -  i get along

Albums Completed:  35
Elapsed Time:  141.4524440765381


Albums Completed:  40
Elapsed Time:  160.89457201957703


Albums Completed:  45
Elapsed Time:  191.2106227874756


Albums Completed:  50
Elapsed Time:  222.4163579940796

Album Not Found:  hasidic new wave and yakar rhythms  -  from the belly of abraham
Album Not Found


Albums Completed:  470
Elapsed Time:  1926.0764918327332

Album Not Found:  weird wives  -  some motherfuckers gonna be walking round with a size 9 diehard up their ass

Albums Completed:  475
Elapsed Time:  1950.1853070259094


Albums Completed:  480
Elapsed Time:  1969.03000998497

Album Not Found:  blonde redhead  -  melodie citronique

Albums Completed:  485
Elapsed Time:  1989.2951610088348


Albums Completed:  490
Elapsed Time:  2023.010909795761

Album Not Found:  lesser  -  c64 massive sidplay party mix cd

Albums Completed:  495
Elapsed Time:  2048.707097053528

Album Not Found:  aluminium group  -  pelo

Albums Completed:  500
Elapsed Time:  2061.9834830760956


Albums Completed:  505
Elapsed Time:  2081.4651601314545

Album Not Found:  mix master mike  -  bangzilla

Albums Completed:  510
Elapsed Time:  2107.819859981537

Album Not Found:  creepoid  -  cemetery highrise slum

Albums Completed:  515
Elapsed Time:  2125.533212184906


Albums Completed:  520
Elapsed Time:  214

Album Not Found:  digitalism  -  kitsun tabloid

Albums Completed:  925
Elapsed Time:  3733.672214984894


Albums Completed:  930
Elapsed Time:  3758.50341296196

Album Not Found:  the go team  -  audio assault course
Album Not Found:  six finger satellite  -  a good year for hardness

Albums Completed:  935
Elapsed Time:  3776.4448750019073


Albums Completed:  940
Elapsed Time:  3790.990294933319


Albums Completed:  945
Elapsed Time:  3816.7100219726562


Albums Completed:  950
Elapsed Time:  3832.5638539791107

Album Not Found:  guided by voices  -  hold on hope
Album Not Found:  flickerstick  -  welcoming home the astronauts

Albums Completed:  955
Elapsed Time:  3851.6317789554596

Album Not Found:  ulrich schnauss  -  stars

Albums Completed:  960
Elapsed Time:  3865.28382897377


Albums Completed:  965
Elapsed Time:  3887.2346217632294

Album Not Found:  various artists  -  big change songs for finca

Albums Completed:  970
Elapsed Time:  3913.617926120758


Albums Completed:  


Albums Completed:  1375
Elapsed Time:  5587.664359092712


Albums Completed:  1380
Elapsed Time:  5609.4776129722595

Album Not Found:  lil wayne  -  dedication 3

Albums Completed:  1385
Elapsed Time:  5630.8090279102325


Albums Completed:  1390
Elapsed Time:  5657.202639102936


Albums Completed:  1395
Elapsed Time:  5677.326596021652

Album Not Found:  quixotic  -  mortal mirror

Albums Completed:  1400
Elapsed Time:  5694.760380983353


Albums Completed:  1405
Elapsed Time:  5711.3954520225525

Album Not Found:  joan of arc  -  presents guitar duets
Album Not Found:  kelley stoltz  -  the past was faster

Albums Completed:  1410
Elapsed Time:  5731.687273979187

Album Not Found:  yasiin bey  -  december 99th
Album Not Found:  the mars volta  -  scab dates

Albums Completed:  1415
Elapsed Time:  5742.62041592598

Album Not Found:  electronicat  -  21st century toy

Albums Completed:  1420
Elapsed Time:  5757.1847448349


Albums Completed:  1425
Elapsed Time:  5782.504019975662

Al

Album Not Found:  black rebel motorcycle club  -  the effects of 333

Albums Completed:  1815
Elapsed Time:  7313.041167020798

Album Not Found:  wolfie  -  wheres wolfie

Albums Completed:  1820
Elapsed Time:  7340.286724090576


Albums Completed:  1825
Elapsed Time:  7361.312566041946



In [191]:
print('Found {}% of Bottom 10% Pitchfork Albums on Spotify'.format(round(100-len(missing_bottom_album_indices)/1839*100, 2)))

Found 88.42% of Bottom 10% Pitchfork Albums on Spotify


In [32]:
len(bottom_albums) - len(missing_bottom_album_indices)

1616

In [33]:
integer_conv_cols = ['song_duration', 'song_explicit', 'song_track_number', 'time_signature',
                     'mode', 'key', 'popularity']

for col in integer_conv_cols:
    top_album_songs[col] = top_album_songs[col].astype(int)
    bottom_album_songs[col] = bottom_album_songs[col].astype(int)

In [34]:
top_album_songs['release_date'] = pd.to_datetime(top_album_songs['release_date'])
bottom_album_songs['release_date'] = pd.to_datetime(top_album_songs['release_date'])

In [35]:
top_album_songs = top_album_songs.merge(genre_ohe_df, how='inner', on='reviewid')
bottom_album_songs = bottom_album_songs.merge(genre_ohe_df, how='inner', on='reviewid')

# Scrape Song Lyrics from Genius
### Enter Genius Credentials
Note:  Delete before comitting

In [36]:
GENIUS_CLIENT_ID = '_VW4p-Rdc-EXszsX0eIyyjSby7GwRjVwdIYgwjiZcg6_DnUfIrNlurBRdPsXeW7-'
GENIUS_API_SECRET = 'h3x5KrkdfDIIvO-n2Psck1hrCoKYmN22q_ovDqN6DLhQDt3KarSaRhXYkxLTpjLuMNEYqxZcfNXG3RgQ_iTrLg'
GENIUS_API_TOKEN = 'qbuj8ynH5Vfvsl8wfDwSSfaVYRF6I36vnZKCwqb37xjG1ASLDRZRtKZd8HFDLuPb'

In [163]:
def clean_lyrics_query(name):
    clean_name = ''.join([x if ord(x) != 160 else ' ' for x in name])    
    clean_name = clean_name.lower()
    clean_name = re.sub('ö', 'o', clean_name)
    clean_name = re.sub("[\(\[].*?[\)\]]", "", clean_name).strip()
    clean_name = re.sub(' & ', ' and ', clean_name)
    clean_name = re.sub('-', ' ', clean_name)
    clean_name = re.sub("[,.']", '', clean_name)
    clean_name = re.sub(' +', ' ', clean_name)
    clean_name = re.sub('"', '', clean_name)
    clean_name = re.sub('!', '', clean_name)
    clean_name = re.sub(':', '', clean_name)
    clean_name = re.sub('/', ' ', clean_name)
    clean_name = re.sub(r'[^0-9A-Za-z ]', '', clean_name)
    clean_name = re.sub("o clock", "oclock", clean_name)
    clean_name = clean_name.strip()
    
    if clean_name[-3:] == ' ep':
        clean_name = clean_name[:-3]
        
    if clean_name[-4:] == ' ost':
        clean_name = clean_name[:-4]
        
    return clean_name

In [172]:
def song_search(song_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + GENIUS_API_TOKEN}
    search_url = base_url + '/search?per_page=50&page=' + str(page)
    data = {'q': song_name}
    response = requests.get(search_url, data=data, headers=headers)
    return response

def request_song_url(song_name, artist_name, album_name, song_cap, page_cap):
    page = 1
    songs = []
    artist_name_query = clean_lyrics_query(artist_name)
    album_name_query = clean_lyrics_query(album_name)
    song_name_query = clean_lyrics_query(song_name)
    
    while True:
        if artist_name == 'various artists':
            search_query = song_name_query + ' ' + album_name_query
        else:
            search_query = song_name_query + ' ' + artist_name_query

        response = song_search(search_query, page)
        if 'json' in response.headers.get('Content-Type'):
            json = response.json()

            # Collect up to song_cap song objects from artist
            song_info = []
            for hit in json['response']['hits']:

                search_result_artist = clean_lyrics_query(hit['result']['primary_artist']['name'])
                search_result_song = clean_lyrics_query(hit['result']['title'])

                if (artist_name != 'various artists'):
                    if (artist_name_query in search_result_artist) and (search_result_song in song_name_query):
                        song_info.append(hit)

            # Collect song URLs from song objects
            for song in song_info:
                if (len(songs) < song_cap):
                    url = song['result']['url']
                    songs.append(url)

            if (len(songs) == song_cap) or (page == page_cap):
                break
            else:
                page += 1
        else:
            print('Response content not in JSON format.')
    
    return songs

In [179]:
def scrape_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    if html.find('div', class_='lyrics') != None:
        lyrics = html.find('div', class_='lyrics').get_text()
    
        # remove parenthetical identifiers
        lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)

        # remove empty lines
        lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])
    else:
        lyrics = 'lyrics not found'
    
    return lyrics

In [39]:
top_album_songs['song_lyrics'] = 'lyrics not found'
bottom_album_songs['song_lyrics'] = 'lyrics not found'

In [40]:
def add_lyrics(df):
    for df_index in range(len(df)):
        if df_index % 50 == 0:
            print('{}% Completed'.format(round(100*df_index/len(df), 1)))
        artist_lookup = df.iloc[df_index]['artist_name']
        song_lookup = clean_name(df.iloc[df_index]['song_name'])
        album_lookup = df.iloc[df_index]['album_title']
        songs = request_song_url(song_lookup, artist_lookup, album_lookup, 1, 1)
        
        if len(songs) == 0:
            truncated_song_lookup = song_lookup
            while len(truncated_song_lookup.split(' ')) > 1:
                truncated_song_lookup = ' '.join(truncated_song_lookup.split(' ')[:-1])
                songs = request_song_url(truncated_song_lookup, artist_lookup, album_lookup, 1, 1)
                if len(songs) > 0:
                    break
            
        if (len(songs) > 0):
            df.iloc[df_index, df.columns.get_loc('song_lyrics')] = scrape_lyrics(songs[0])
        else:
            print('lyrics not found: {} - {}'.format(df.iloc[df_index]['artist_name'], 
                                                     df.iloc[df_index]['song_name'].split('-')[0]))
        
    return df
            

In [176]:
top_album_songs = add_lyrics(top_album_songs)

0.0% Completed
0.2% Completed
lyrics not found: john coltrane - A Love Supreme, Pt. I – Acknowledgement
lyrics not found: john coltrane - A Love Supreme, Pt. II 
lyrics not found: john coltrane - A Love Supreme, Pt. III 
lyrics not found: john coltrane - A Love Supreme, Pt. IV 
lyrics not found: john coltrane - A Love Supreme, Pt. III 
lyrics not found: john coltrane - A Love Supreme, Pt. IV 
lyrics not found: john coltrane - A Love Supreme, Pt. I 
lyrics not found: john coltrane - A Love Supreme, Pt. I 
lyrics not found: john coltrane - A Love Supreme Pt. II 
lyrics not found: john coltrane - A Love Supreme Pt. II 
lyrics not found: john coltrane - A Love Supreme, Pt. IV 
lyrics not found: john coltrane - A Love Supreme Pt. I 
lyrics not found: john coltrane - A Love Supreme Pt. I 
lyrics not found: john coltrane - A Love Supreme Pt. I 
lyrics not found: john coltrane - A Love Supreme Pt. I 
lyrics not found: john coltrane - A Love Supreme Pt. I 
lyrics not found: john coltrane - A Lo

lyrics not found: mobb deep - (Just Step Prelude)
lyrics not found: mobb deep - Q.U. 
lyrics not found: mobb deep - (The Grave Prelude)
lyrics not found: beastie boys - B
lyrics not found: beastie boys - B
lyrics not found: beastie boys - B
lyrics not found: beastie boys - B
lyrics not found: beastie boys - B
lyrics not found: beastie boys - B
lyrics not found: beastie boys - B
lyrics not found: beastie boys - B
lyrics not found: beastie boys - B
4.0% Completed
4.2% Completed
lyrics not found: serge gainsbourg - Ah Melody
lyrics not found: the beach boys - I Wanna Be Around/Workshop
4.4% Completed
4.6% Completed
lyrics not found: nirvana - Forgotten Tune 
lyrics not found: nirvana - Jam 
4.7% Completed
lyrics not found: neutral milk hotel - King of Carrot Flowers Pt. 1
lyrics not found: neutral milk hotel - King of Carrot Flowers Pts. 2 & 3
4.9% Completed
5.1% Completed
lyrics not found: elvis costello and the attractions - No Action
lyrics not found: elvis costello and the attractions

lyrics not found: rem - 1,000,000 
lyrics not found: rem - Theme From Two Steps Onward 
6.3% Completed
lyrics not found: arcade fire - Une Annee Sans Lumiere
6.5% Completed
lyrics not found: keith fullerton whitman - Track3a (2waynice)
lyrics not found: various artists - Ask The Lonely 
lyrics not found: various artists - Where Did You Go
lyrics not found: various artists - Do You Know What I'm Talkin' About
lyrics not found: various artists - You Say You Love Me
lyrics not found: various artists - Conscience I'm Guilty
lyrics not found: various artists - You Got Me Worried
lyrics not found: various artists - Shotgun
lyrics not found: various artists - Hot Cha
lyrics not found: various artists - All For You
lyrics not found: various artists - Too Many Fish In The Sea
lyrics not found: various artists - Love Makes The World Go `Round, But Money Greases The Wheel
lyrics not found: various artists - Come On Back (And Be My Love Again)
lyrics not found: various artists - I'm Still Loving Y

lyrics not found: miles davis - Directions 
lyrics not found: miles davis - Directions 
lyrics not found: farben - Live At The Sahara Tahoe, 1973 
lyrics not found: farben - Love To Love You Baby 
lyrics not found: farben - Suntouch Edit 
9.1% Completed
lyrics not found: farben - Beautone 
lyrics not found: farben - So Much Love 
lyrics not found: farben - Bayreuth 
lyrics not found: farben - FF 
lyrics not found: farben - Silikon 
lyrics not found: farben - Love Oh Love 
lyrics not found: interpol - Get The Girls/Song 5 (First Demo)
lyrics not found: interpol - Get The Girls/Song 5 (First Demo)
9.3% Completed
lyrics not found: morton feldman ives ensemble - String Quartet (II) (1983): I.
lyrics not found: morton feldman ives ensemble - String Quartet (II) (1983): II.
lyrics not found: morton feldman ives ensemble - String Quartet (II) (1983): III.
lyrics not found: morton feldman ives ensemble - String Quartet (II) (1983): IV.
lyrics not found: morton feldman ives ensemble - String Qu

lyrics not found: various artists - (We've Got) Honey Love
lyrics not found: various artists - I'm In Love (And I Know It) 
lyrics not found: various artists - Runaway Child, Running Wild 
lyrics not found: various artists - Gonna Give Her All The Love I've Got 
lyrics not found: various artists - For Better Or Worse 
lyrics not found: various artists - Don't Mess With Bill
lyrics not found: various artists - The Composer 
lyrics not found: various artists - The Beginning Of The End 
lyrics not found: various artists - My Cup Runneth Over
lyrics not found: various artists - Ask The Lonely
lyrics not found: various artists - Too Busy Thinking About My Baby 
lyrics not found: various artists - Wherever I Lay My Hat (That's My Home)
lyrics not found: various artists - What Is A Man
lyrics not found: various artists - Don't Bring Back Memories
lyrics not found: various artists - Cherie 
lyrics not found: various artists - Baby 
lyrics not found: various artists - What Does It Take (To Win 

...

lyrics not found: philippe besombes - Les cosmonautes
lyrics not found: philippe besombes - Avécandista
lyrics not found: craig leon - Ring with Three Concentric Discs
lyrics not found: craig leon - Donkeys Bearing Cups
lyrics not found: craig leon - Nommo
90.4% Completed
lyrics not found: craig leon - Four Eyes to See the Afterlife
lyrics not found: craig leon - She Wears a Hemispherical Skullcap
lyrics not found: craig leon - One Hundred Steps
lyrics not found: craig leon - Region of Fleeing Civilians
lyrics not found: craig leon - Three Small Coins
lyrics not found: craig leon - Visiting
lyrics not found: craig leon - Details Suggest Fidelity to Fact
lyrics not found: craig leon - The Customs of the Age Disturbed
lyrics not found: nomo - Nu Tones
lyrics not found: nomo - Hand and Mouth
lyrics not found: nomo - Fourth Ward
lyrics not found: nomo - Reasons
lyrics not found: nomo - New Song
lyrics not found: nomo - Divisions
lyrics not found: nomo - We Do We Go
lyrics not found: nomo -

lyrics not found: various artists - Travis
lyrics not found: various artists - Fly Again
lyrics not found: various artists - Give In to Me
lyrics not found: various artists - Hide Me Babe
lyrics not found: various artists - Timing Is Everything
93.4% Completed
lyrics not found: wovenhand - Story and Pidtures
lyrics not found: pinetop seven - The Night's Bloom
lyrics not found: pinetop seven - Fringe
lyrics not found: pinetop seven - A Fire Burns At The Foot Of Our Bed
lyrics not found: pinetop seven - Born Among The Born Again
lyrics not found: pinetop seven - A Mouthful Of Expensive Teeth
lyrics not found: pinetop seven - June
lyrics not found: pinetop seven - Untitled
lyrics not found: pinetop seven - The Comedy Is Ended
lyrics not found: migala - La Canción de Gurb
lyrics not found: migala - El Pasado Diciembre
lyrics not found: migala - Noche Desde un Tren
lyrics not found: migala - El Retraso
lyrics not found: migala - Ciudad del Oeste
93.6% Completed
lyrics not found: migala - Un

lyrics not found: esg - Talk It
lyrics not found: esg - Hold Me Right
lyrics not found: esg - It's Alright
lyrics not found: esg - Get Funky
lyrics not found: esg - I Can't Tell You What To Do
lyrics not found: esg - In The Streets
lyrics not found: esg - About You
lyrics not found: esg - Insane 
lyrics not found: esg - Keep On Moving
lyrics not found: esg - Dance To The Beat Of Moody
lyrics not found: esg - Bam Bam Jam
lyrics not found: esg - A New Day
lyrics not found: esg - Standing In Line
lyrics not found: esg - I Wanna Dance
lyrics not found: esg - Six Pack 
lyrics not found: esg - There Was A Time
lyrics not found: esg - Earn It
lyrics not found: esg - Like This
lyrics not found: sam prekop - Little Bridges
lyrics not found: sam prekop - A Splendid Hollow
lyrics not found: sam prekop - C+F
lyrics not found: sam prekop - Density
96.2% Completed
lyrics not found: goldfrapp - A&E
lyrics not found: the soft pink truth - Everybody's soft
lyrics not found: the soft pink truth - Gender

lyrics not found: white williams - In The Club
lyrics not found: white williams - Going Down
lyrics not found: white williams - Lice In The Rainbow
lyrics not found: white williams - Violator
lyrics not found: white williams - Smoke
lyrics not found: white williams - The Shadow
lyrics not found: white williams - Danger
lyrics not found: white williams - Route To Palm
lyrics not found: black cat music - The Chain
lyrics not found: black cat music - Williamsburg Bridge Song
lyrics not found: black cat music - The Princess Dress
lyrics not found: black cat music - Rats Bats Translucent Blood
lyrics not found: black cat music - At the End of All That's Good and Pure
97.9% Completed
lyrics not found: shangaan electro - Ngunyuta Dance
lyrics not found: shangaan electro - Nwampfundla
lyrics not found: shangaan electro - Vana Vasesi
lyrics not found: shangaan electro - Ntombi Ya Mugaza
lyrics not found: shangaan electro - N'wagezani My Love
lyrics not found: shangaan electro - Vanghoma
lyrics 

lyrics not found: appendix out - Organise a March
lyrics not found: tujiko noriko - Narita Made
lyrics not found: tujiko noriko - Zipper
lyrics not found: tujiko noriko - Rocket Hanabi
lyrics not found: tujiko noriko - Mugen Kyuukou
lyrics not found: tujiko noriko - Kiminotamen
lyrics not found: tujiko noriko - Tokyo
lyrics not found: tujiko noriko - Tokyo Power
lyrics not found: tujiko noriko - Robot Hero
lyrics not found: gza - Intro
lyrics not found: gza - Fam (Members Only) 
lyrics not found: my morning jacket - Mahgeeta
lyrics not found: my morning jacket - Masterplan
99.7% Completed
lyrics not found: deerhoof - Adam+Eve Connection
lyrics not found: shocking pinks - Wake Up
lyrics not found: shocking pinks - How Am I Not Myself?
lyrics not found: shocking pinks - Second Hand Girl
lyrics not found: shocking pinks - End of the World
lyrics not found: shocking pinks - The Narrator
lyrics not found: shocking pinks - Yes! No!
lyrics not found: shocking pinks - Emily
lyrics not found: s

In [180]:
bottom_album_songs = add_lyrics(bottom_album_songs)

0.0% Completed
lyrics not found: icarus - UL
lyrics not found: icarus - Borichean Pintak
lyrics not found: icarus - Deutsche Oper
lyrics not found: icarus - Benevolent Incubator
lyrics not found: icarus - Despair
lyrics not found: icarus - Nine Fresian
lyrics not found: icarus - Dolphin Lylic
lyrics not found: icarus - Kipperkun Scratch
lyrics not found: icarus - Xot Zoiks
lyrics not found: icarus - Skate Glug
lyrics not found: icarus - Elephant Kiosk
lyrics not found: jessica lea mayfield seth avett - Between the Bars
lyrics not found: jessica lea mayfield seth avett - Baby Britain
lyrics not found: jessica lea mayfield seth avett - Fond Farewell
0.2% Completed
lyrics not found: jessica lea mayfield seth avett - Somebody That I Used to Know
lyrics not found: jessica lea mayfield seth avett - Let's Get Lost
lyrics not found: jessica lea mayfield seth avett - Twilight
lyrics not found: jessica lea mayfield seth avett - Ballad of Big Nothing
lyrics not found: jessica lea mayfield seth av

lyrics not found: yoshimi and yuka - Korokokoro'n Insects
lyrics not found: yoshimi and yuka - Spy Said One
lyrics not found: yoshimi and yuka - La Donna Ni Demo Des Kinna
lyrics not found: yoshimi and yuka - Mow Deck in Eye
lyrics not found: yoshimi and yuka - Elegant Bird
lyrics not found: blight - Blight
lyrics not found: blight - Dream Is Dead
lyrics not found: blight - Bludgeon
lyrics not found: blight - Prophet of Doom
lyrics not found: blight - Real World
lyrics not found: blight - Be Stupid
lyrics not found: blight - Girlie Girlie
lyrics not found: blight - Armageddon
lyrics not found: blight - Seven Winds Over the Gobi Desert
lyrics not found: blight - Blight 
lyrics not found: blight - Be Stupid 
lyrics not found: blight - Prophet of Doom 
lyrics not found: blight - Real World 
lyrics not found: blight - Be Stupid 
lyrics not found: blight - Bludgeon 
lyrics not found: blight - Armageddon 
lyrics not found: blight - Girlie Girle 
lyrics not found: blight - Dream Is Dead 
lyri

lyrics not found: zombi - Spirit Animal
lyrics not found: zombi - Earthly Powers
lyrics not found: zombi - Cosmic Powers
lyrics not found: zombi - Through Time
lyrics not found: discodeine - Falkenberg
lyrics not found: discodeine - DA
4.9% Completed
lyrics not found: discodeine - Antiphonie
lyrics not found: discodeine - Ring Mutilation
lyrics not found: discodeine - Depression Skit
lyrics not found: discodeine - Grace
lyrics not found: discodeine - HomoCompatible
lyrics not found: discodeine - Relaps
lyrics not found: discodeine - Invert
lyrics not found: discodeine - Figures In A Soundscape
lyrics not found: discodeine - Tom Select 
lyrics not found: clem snide - The Meat Of Life
lyrics not found: clem snide - With Nothing To Show For it
lyrics not found: clem snide - Anita
lyrics not found: ogurusu norihide - 7:07
lyrics not found: ogurusu norihide - 5:01
lyrics not found: ogurusu norihide - 2:45
lyrics not found: ogurusu norihide - 4:10
lyrics not found: ogurusu norihide - 6:28
ly

lyrics not found: say hi to your mom - Snowcones and Puppies
lyrics not found: say hi to your mom - Blah Blah Blah
lyrics not found: say hi to your mom - Sad, But Endearingly So
lyrics not found: say hi to your mom - She Just Happens to Date the Prince of Darkness
lyrics not found: say hi to your mom - Prefers Unhappy Endings
lyrics not found: say hi to your mom - Angels and Darlas
lyrics not found: say hi to your mom - Not As Goth As They Say We Are
lyrics not found: say hi to your mom - Sweet Sweet Heartkiller
lyrics not found: say hi to your mom - The Reigning Champ of the Teething Crowd
6.6% Completed
lyrics not found: guided by voices - Trash Truck
lyrics not found: guided by voices - Troppers in the Town
lyrics not found: devin the dude - Cutural Coughee
6.8% Completed
lyrics not found: devin the dude - I Gotta Ho
lyrics not found: devin the dude - Ultimate High
lyrics not found: devin the dude - It's On You
lyrics not found: devin the dude - All You Need
lyrics not found: murder

lyrics not found: ex cops - Modern World
lyrics not found: ex cops - Tragically Alright
lyrics not found: ex cops - Rooms
lyrics not found: ex cops - Weird With You
lyrics not found: otto von schirach - 25 Degress 46' X 80 Degrees 12'
lyrics not found: otto von schirach - Time Traveling Lives
lyrics not found: otto von schirach - Tympanic Calcoolus
lyrics not found: otto von schirach - Lunatic Nitrates
lyrics not found: otto von schirach - C21 H39 N7 O12
lyrics not found: otto von schirach - Triangle Exit
lyrics not found: otto von schirach - Ecleptoze Chemiz
lyrics not found: otto von schirach - Endothermic Cavewalker
lyrics not found: otto von schirach - Schematropolis Mirage
lyrics not found: otto von schirach - No Wood
lyrics not found: otto von schirach - Aquantumzation Below Zero
lyrics not found: otto von schirach - Elastic Paranormalites
lyrics not found: otto von schirach - Insectdezyde Juice
lyrics not found: otto von schirach - Smelly Mustard
lyrics not found: otto von schir

...

88.8% Completed
lyrics not found: turbonegro - Hot Stuff/Hot Shit
lyrics not found: d4 - Rock 'n' Roll Motherfucker
lyrics not found: d4 - Running on Empty
lyrics not found: d4 - Ladies Man
lyrics not found: d4 - Invader Ace
lyrics not found: d4 - Mysterex
lyrics not found: red snapper - Regrettable
lyrics not found: red snapper - Mountains and Valleys
lyrics not found: red snapper - Ultraviolet
lyrics not found: red snapper - Heavy Petting
lyrics not found: red snapper - Dnipro
lyrics not found: red snapper - Hot Flush (Sabres of Paradise remix)
lyrics not found: red snapper - Odd Man out
lyrics not found: red snapper - The Quiet One
lyrics not found: red snapper - The Tunnel (live)
lyrics not found: red snapper - 4 Dead Monks (live)
lyrics not found: arling and cameron - 5th Dimension
lyrics not found: arling and cameron - B.B. Electro
lyrics not found: arling and cameron - Up
lyrics not found: arling and cameron - We Are a & C
lyrics not found: arling and cameron - Freedom, Right No

lyrics not found: the jon spencer blues explosion - Over & Over
lyrics not found: the jon spencer blues explosion - Mother Nature
lyrics not found: the jon spencer blues explosion - Mean Heart
lyrics not found: the jon spencer blues explosion - Point Of View
lyrics not found: mommy and daddy - Confection
lyrics not found: mommy and daddy - Street Cleanor Demeanor
lyrics not found: mommy and daddy - The Now
lyrics not found: mommy and daddy - Run It Off
lyrics not found: mommy and daddy - Question Marks Followed by Exclamation Marks
lyrics not found: mommy and daddy - Confection (Extended)
lyrics not found: bravehearts - Intro 
lyrics not found: bravehearts - For The Love Interlude 
lyrics not found: bravehearts - Freak Off Interlude 
91.5% Completed
lyrics not found: iggy pop - Sterility
lyrics not found: steve aoki - Waters Of Nazareth 
lyrics not found: steve aoki - We Are Rockstars 
lyrics not found: steve aoki - Element Of Danger 
lyrics not found: steve aoki - Bring It On 
lyrics 

lyrics not found: various artists - Never See You
lyrics not found: various artists - It's Alright to Fail
lyrics not found: various artists - One Day the Dam Will Break
lyrics not found: various artists - You, The Queen
lyrics not found: various artists - Morning in a Strange City (Cafe)
lyrics not found: various artists - No More
lyrics not found: various artists - Dear Dorothy
lyrics not found: various artists - Never See You
lyrics not found: various artists - There Are No Good Second Chances
lyrics not found: joan of arc - Leaving Needn't Take Long.
lyrics not found: joan of arc - Ne Mosquitos Pass.
lyrics not found: joan of arc - We Neither Hide Nor Seek,
lyrics not found: joan of arc - Most At Home In Motels.
lyrics not found: joan of arc - My Fight Is Necessary.
lyrics not found: joan of arc - I'll Show You, I'll Show You All.
93.7% Completed
lyrics not found: the entrance band - Lookout!
lyrics not found: the entrance band - Still Be There
lyrics not found: the entrance band -

lyrics not found: various artists - Float On
lyrics not found: various artists - I Turn My Camera On
lyrics not found: various artists - Pretty Vacant
lyrics not found: various artists - California
lyrics not found: various artists - Wasted
lyrics not found: various artists - Can't Get It out of My Head
lyrics not found: various artists - Debaser
lyrics not found: various artists - Hello Sunshine
lyrics not found: various artists - Smile Like You Mean It
lyrics not found: various artists - Come into Our Room
lyrics not found: various artists - The End's Not Near
lyrics not found: various artists - Into Dust
96.4% Completed
lyrics not found: the ting tings - One By One
96.6% Completed
lyrics not found: the ting tings - Ain't Got Shit
lyrics not found: firesign theatre - The Undermutter Moment
lyrics not found: firesign theatre - Pinata! Pinata!
lyrics not found: firesign theatre - S**t Happens!
lyrics not found: the warlocks - It's Just Like Surgery
lyrics not found: the warlocks - Gyps

lyrics not found: shat - Give Me Pussy And Tits And Ass With A Beer
lyrics not found: shat - Fat Attack
lyrics not found: shat - I Smell Fish
lyrics not found: shat - Grandpa's Playing With His Penis
lyrics not found: shat - Fat Ho
lyrics not found: shat - Realization
lyrics not found: shat - Cuntchuck
lyrics not found: shat - Turn Off The Fuckin' T.V.
lyrics not found: shat - Sex With Your Pets
lyrics not found: shat - Close/Open
lyrics not found: shat - Old Fashioned Whore
lyrics not found: shat - Friends Don't Let Friends Drive Drunk
lyrics not found: shat - Breakfast With James Hetfield
lyrics not found: shat - Fuck Just To Fuck
lyrics not found: shat - Kill Baby
lyrics not found: dan le sac vs scroobius pip - Beat That My Heart Skipped
lyrics not found: vue - Nothing Left But You
lyrics not found: vue - Angel's Alright
lyrics not found: vue - We Were Here
lyrics not found: vue - Cotton Kisses
lyrics not found: vue - Her Moods
lyrics not found: vue - I'm Everybody's Funny
lyrics no

In [181]:
top_album_songs.loc[top_album_songs['song_lyrics'] == '', 'song_lyrics'] = 'lyrics not found'
bottom_album_songs.loc[bottom_album_songs['song_lyrics'] == '', 'song_lyrics'] = 'lyrics not found'

In [187]:
top_album_songs.to_csv('top_album_songs.csv', index=False)
bottom_album_songs.to_csv('bottom_album_songs.csv', index=False)

top_albums.to_csv('top_albums.csv', index=False)
bottom_albums.to_csv('bottom_albums.csv', index=False)

scores.to_csv('album_scores.csv', index=False)
genre_ohe_df.to_csv('album_genres.csv', index=False)

top_albums.iloc[missing_top_album_indices].to_csv('missing_top_albums.csv', index=False)
bottom_albums.iloc[missing_bottom_album_indices].to_csv('missing_bottom_albums.csv', index=False)